In [1]:
import pandas as pd
import json

from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

In [45]:
# helper functions


def tokenize_function(examples):
    
    return tokenizer(examples["summary"], padding = "max_length", truncation = True)

In [38]:
data_files = {
        "train" : "../datasets/training",
        "val" : "../datasets/validation",
        "test" : "../datasets/test",
        } 

training = load_dataset("json", data_files = data_files, split = "train")
validation = load_dataset("json", data_files = data_files, split = "val")
test = load_dataset("json", data_files = data_files, split = "test")

In [55]:
training['title'][-1]

'Occupant'

In [51]:
# training = load_dataset("json", data_files="../datasets/training", split = "train")
# validation = load_dataset("json", data_files="../datasets/validation", split = "val")
# test = load_dataset("json", data_files="../datasets/test", split = "test")

# for i, ex in enumerate(training["train"], start = 1):
    
#     print(ex)
#     print()
    
#     if (i == 10):
        
#         break

In [ ]:
# load them using hugging face
# then map the tokenizer across them

##### Train using PyTorch Trainer API

In [48]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

In [64]:
tokenizer(movies_df["summary"][0], padding = "max_length", truncation = True)

{'input_ids': [101, 159, 1663, 8863, 1204, 5561, 2736, 1176, 170, 1873, 1133, 1144, 4743, 1939, 1104, 1739, 119, 8258, 2824, 1200, 1414, 1874, 4090, 1123, 1223, 8415, 2615, 1105, 2274, 1123, 1313, 119, 1414, 1874, 1105, 1117, 1676, 16452, 1162, 4958, 1106, 1712, 159, 1663, 8863, 1204, 5561, 1105, 4693, 1123, 1176, 1126, 6655, 1873, 119, 1252, 159, 1663, 8863, 1204, 5561, 2762, 112, 189, 1126, 6655, 1873, 1105, 1131, 10498, 1283, 119, 1153, 3349, 1106, 1301, 1106, 1103, 1375, 1114, 1103, 1168, 4939, 119, 1135, 3316, 170, 1263, 1105, 16889, 25576, 5012, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [61]:
movies_df["summary"].map(lambda x: tokenizer(x, padding = "max_length", truncation = True))

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [44]:
tokenized_datasets = movies_df.map(tokenize_function, batched = True)

AttributeError: 'DataFrame' object has no attribute 'map'